In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# from open_ephys.analysis import Session as OESession
from pathlib import Path
from functools import partial
from glob import glob
import pyarrow
import plotly.express as px
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler
from plotly.subplots import make_subplots

from projectlib.utils import read_3d_data, sns_setup, VideoFrames
from projectlib.anatomy import compute_measurements_df

sns_setup(font="sans-serif")

In [3]:
# load amd inspect behavioral data 

meas_df = pd.read_pickle("fe-data/meas_df.pkl")
meas_df

,date,mouse,run,measurement_group,measurement_name,measurement_units,measurement_type,measurement_value,std,count,timeseries
0,20240815,M3,000,eye,eye-height-left,mm,distance,2.356229,0.056407,73259,"[2.385401718146344, 2.399465843066115, 2.40885..."
14,20240815,M3,000,mouth,mouth-area,mm^2,area,2.365503,1.319462,73259,"[1.8182747756839102, 1.8214548810665163, 1.828..."
13,20240815,M3,000,ear,ear-area-right,mm^2,area,54.502883,6.956509,73259,"[55.224244767779936, 55.2714536194955, 55.2668..."
12,20240815,M3,000,ear,ear-area-left,mm^2,area,60.157882,3.198757,73259,"[55.57828268916887, 56.43301136572061, 57.2849..."
11,20240815,M3,000,ear,ear-angle-right,deg,angle,139.532192,14.014073,73259,"[145.02778632674986, 145.012809332967, 144.985..."
10,20240815,M3,000,ear,ear-angle-left,deg,angle,161.454108,3.007539,73259,"[156.26229084391233, 156.71180202194583, 157.0..."
9,20240815,M3,000,ear,ear-width-right,mm,distance,5.098862,0.516213,73259,"[5.061174738366927, 5.041033999827993, 5.02219..."
15,20240815,M3,000,nose,nose-bulge-volume,mm^3,volume,9.529116,0.694226,73259,"[8.877387667884895, 9.205450785961167, 9.45941..."
8,20240815,M3,000,ear,ear-width-left,mm,distance,5.758315,0.404739,73259,"[5.193688834047411, 5.3521941919322815, 5.4890..."
6,20240815,M3,000,ear,ear-height-left,mm,distance,11.469678,0.287990,73259,"[11.36328203918898, 11.373430311835518, 11.382..."


In [4]:
# load amd inspect electrophysiology (myomatrix) data 

ephys_df = pd.read_parquet("ephys-data/ephys_df.parquet", engine='pyarrow')
ephys_df

,date,mouse,run,channel,signal,lag_time,sample_rate,lag_slope
0,20240815,M3,000,0,"[-83.06999695301054, -75.0749972462654, -60.44...",-0.019454,30076.520248,-0.002555
1,20240815,M3,000,1,"[-85.40999686717986, -86.18999683856963, -73.1...",-0.019454,30076.520248,-0.002555
2,20240815,M3,000,2,"[-63.37499767541885, -48.359998226165764, -30....",-0.019454,30076.520248,-0.002555
3,20240815,M3,000,3,"[-36.464998662471764, -19.304999291896817, -9....",-0.019454,30076.520248,-0.002555
4,20240815,M3,000,4,"[-26.714999020099636, -22.034999191761013, -5....",-0.019454,30076.520248,-0.002555
5,20240815,M3,000,5,"[-40.169998526573174, -31.199998855590817, -22...",-0.019454,30076.520248,-0.002555
6,20240815,M3,000,6,"[-19.304999291896817, -15.989999413490294, -9....",-0.019454,30076.520248,-0.002555
7,20240815,M3,000,7,"[-22.4249991774559, -20.27999925613403, -18.71...",-0.019454,30076.520248,-0.002555
8,20240815,M3,000,8,"[-81.70499700307845, -76.04999721050261, -65.1...",-0.019454,30076.520248,-0.002555
9,20240815,M3,000,9,"[-329.35498791933054, -246.47999095916745, -13...",-0.019454,30076.520248,-0.002555


In [5]:
# merge behavioral and electrophysiology dataframe into a joint dataframe

key_cols = ("date", "mouse", "run")

sub_meas_df = meas_df[[*key_cols, "measurement_name", "timeseries"]].rename(
    columns={"measurement_name": "signal_name"}
)
sub_meas_df = sub_meas_df.assign(lag_time=0.0, sample_rate=100.0)
sub_ephys_df = ephys_df[[*key_cols, "channel", "signal", "lag_time", "sample_rate"]].rename(
    columns={"channel": "signal_name", "signal": "timeseries"}
)
sub_ephys_df = sub_ephys_df.assign(
    signal_name=sub_ephys_df["signal_name"].apply(lambda x: f"channel_{x}")
)
merge_df = pd.concat([sub_meas_df, sub_ephys_df], axis=0)
merge_df

,date,mouse,run,signal_name,timeseries,lag_time,sample_rate
0,20240815,M3,000,eye-height-left,"[2.385401718146344, 2.399465843066115, 2.40885...",0.000000,100.000000
14,20240815,M3,000,mouth-area,"[1.8182747756839102, 1.8214548810665163, 1.828...",0.000000,100.000000
13,20240815,M3,000,ear-area-right,"[55.224244767779936, 55.2714536194955, 55.2668...",0.000000,100.000000
12,20240815,M3,000,ear-area-left,"[55.57828268916887, 56.43301136572061, 57.2849...",0.000000,100.000000
11,20240815,M3,000,ear-angle-right,"[145.02778632674986, 145.012809332967, 144.985...",0.000000,100.000000
10,20240815,M3,000,ear-angle-left,"[156.26229084391233, 156.71180202194583, 157.0...",0.000000,100.000000
9,20240815,M3,000,ear-width-right,"[5.061174738366927, 5.041033999827993, 5.02219...",0.000000,100.000000
15,20240815,M3,000,nose-bulge-volume,"[8.877387667884895, 9.205450785961167, 9.45941...",0.000000,100.000000
8,20240815,M3,000,ear-width-left,"[5.193688834047411, 5.3521941919322815, 5.4890...",0.000000,100.000000
6,20240815,M3,000,ear-height-left,"[11.36328203918898, 11.373430311835518, 11.382...",0.000000,100.000000


In [6]:
from IPython.display import display

def plotly_helper(signal_name_list, df, time_min, time_duration = 20):
 
    fig = make_subplots(rows=len(signal_name_list), cols=1, shared_xaxes=True, vertical_spacing=0.01, x_title='Time (sec)',)

    for ind, signal_name in enumerate(signal_name_list): 
        # print(signal_name)
        data_row = df.loc[merge_df['signal_name'] == signal_name]
        sample_rate = data_row['sample_rate'].values[0]
        lag_time = data_row['lag_time'].values[0]
        data = data_row['timeseries'].values[0][int(time_min*sample_rate + lag_time) : int((time_min + time_duration)*sample_rate + lag_time)]
        data_ts = np.arange(len(data)) / sample_rate + lag_time + time_min
        # print(len(data_row['timeseries'].values[0]))
        # break
        fig.add_trace(go.Scatter(x = data_ts, y = data, name = signal_name), row=ind + 1, col=1)

    fig.update_yaxes(showticklabels=False)
    fig.update_layout(
        plot_bgcolor="white",
        height = 100*len(signal_name_list),
        width = 1200,

    )
    fig = FigureWidgetResampler(fig)
    
    display(fig)
    # return fig

TIME_MIN = 90  # in sec
SIGNAL_NAME_LIST = [
    'eye-area-left', 
    'eye-area-right',
    'ear-angle-left',
    'ear-angle-right',
    'cheek-bulge-volume',
    'mouth-area',
    'channel_11', 
    'channel_15'
    ]

plotly_helper(signal_name_list = SIGNAL_NAME_LIST, df = merge_df, time_min = TIME_MIN)

FigureWidgetResampler({
    'data': [{'name': ('<b style="color:sandybrown">[R' ... 'tyle="color:#fc9944">~0.02</i>'),
              'type': 'scatter',
              'uid': 'a4bbbbc1-599f-4aa9-8c31-09c6ad5103fa',
              'x': array([ 90.  ,  90.01,  90.04, ..., 109.95, 109.98, 109.99]),
              'xaxis': 'x',
              'y': array([7.24976157, 7.24304303, 7.2219882 , ..., 7.23271251, 7.29485818,
                          7.31086873]),
              'yaxis': 'y'},
             {'name': ('<b style="color:sandybrown">[R' ... 'tyle="color:#fc9944">~0.02</i>'),
              'type': 'scatter',
              'uid': 'f1d0ea4e-47ff-42a0-99af-7d91b9776470',
              'x': array([ 90.  ,  90.02,  90.04, ..., 109.95, 109.98, 109.99]),
              'xaxis': 'x2',
              'y': array([5.16889193, 5.17424479, 5.17764332, ..., 5.65294448, 5.6527138 ,
                          5.6508227 ]),
              'yaxis': 'y2'},
             {'name': ('<b style="color:sandybrown">[R' ..

In [7]:
#TODO Troublehsoot: make sure ipywidgets is properly setup

import ipywidgets as widgets
from IPython.display import display

slider = widgets.IntSlider(value=50, min=0, max=100, step=1, description='Test Slider:')
display(slider)

IntSlider(value=50, description='Test Slider:')